In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, GRU, Flatten,Dropout, LSTM 
from keras.optimizers import SGD
from tensorflow.keras.layers import Embedding
from keras.preprocessing import sequence

In [2]:
df=pd.read_csv('/content/GRU_case4_with_pca_with_smote.csv')

In [3]:
print(df)

        Age     Workclass    fnlwgt        education         marital-status  \
0      31.0    ' Private'  157640.0       ' HS-grad'  ' Married-civ-spouse'   
1      44.0    ' Private'  222504.0  ' Some-college'  ' Married-civ-spouse'   
2      52.0    ' Private'  146567.0       ' HS-grad'            ' Divorced'   
3      55.0    ' Private'  143266.0     ' Assoc-voc'  ' Married-civ-spouse'   
4      29.0    ' Private'  278637.0     ' Bachelors'  ' Married-civ-spouse'   
...     ...           ...       ...              ...                    ...   
49405  22.0    ' Private'  310152.0  ' Some-college'       ' Never-married'   
49406  27.0    ' Private'  257302.0    ' Assoc-acdm'  ' Married-civ-spouse'   
49407  58.0    ' Private'  151910.0       ' HS-grad'             ' Widowed'   
49408  22.0    ' Private'  201490.0       ' HS-grad'       ' Never-married'   
49409  39.0  ' State-gov'   77516.0     ' Bachelors'       ' Never-married'   

                 occupation      relationship      

In [4]:
#tranforming categorical data to numerical
from sklearn.preprocessing import LabelEncoder 
labelencoder= LabelEncoder() 
for column in df:
    df[column] = labelencoder.fit_transform(df[column]) 
df.head()

,Age,Workclass,fnlwgt,education,marital-status,occupation,relationship,race,Gender,hours-per-week,native-country,Potential
0,942,3,15182,11,2,9,0,4,1,16260,38,0
1,11841,3,26975,15,2,0,5,4,0,9297,38,0
2,18507,3,13565,11,0,6,1,2,1,2936,38,0
3,20053,3,13075,8,2,2,0,4,1,13826,17,0
4,441,3,31825,9,2,11,0,4,1,9297,38,0


In [5]:
#separating input and output attributes
X = df.iloc[:,:11].values
y = df.iloc[:,11:].values

In [6]:
#splitting into testing and training dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
print(len(X_train),len(X_test),len(y_train),len(y_test))

39528 9882 39528 9882


In [7]:
import tensorflow as tf

In [8]:
from keras.utils import to_categorical
num_classes = 2

y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

print(y_train.shape)
print(y_train[0])
#print(len(y_train[0]))

(39528, 2)
[0. 1.]


In [9]:
# only take 500 words per review
max_words = 500
from keras_preprocessing.sequence import pad_sequences

X_train = pad_sequences (X_train, maxlen=max_words)
X_test = pad_sequences(X_test, maxlen=max_words)

In [10]:
model = Sequential ()
top_words=100
model.add (Embedding (top_words, 100, input_length=max_words)) 
model.add(GRU(100))
model.add(Dense (1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 100)          10000     
                                                                 
 gru (GRU)                   (None, 100)               60600     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 70,701
Trainable params: 70,701
Non-trainable params: 0
_________________________________________________________________
None


In [11]:
training_sentences = []
training_labels = []
testing_sentences = []
testing_labels = []
for s in X_train:
    training_sentences.append(str(s))
for l in X_train:
    training_labels.append(l)
for s in y_test:
    testing_sentences.append(str(s))
for l in y_test:
    testing_labels.append(l)

In [12]:
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [13]:
vocab_size = 10000
embedding_dim=16
max_length = 120
trunc_type= 'post'
oov_tok=""

In [14]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"


tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index

In [15]:
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences, maxlen=max_length,  truncating = trunc_type)
testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length)

In [16]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                             input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32)),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 120, 16)           160000    
                                                                 
 bidirectional (Bidirectiona  (None, 64)               9600      
 l)                                                              
                                                                 
 dense_1 (Dense)             (None, 10)                650       
                                                                 
 dense_2 (Dense)             (None, 1)                 11        
                                                                 
Total params: 170,261
Trainable params: 170,261
Non-trainable params: 0
_________________________________________________________________


In [17]:
#testing_padded = np.asarray(training_labels).astype('float32').reshape((-1,1))
#testing_labels_final= np.asarray(testing_labels).astype('float32').reshape((-1,1))

In [18]:
import tensorflow as tf

testing_padded = tf.stack(testing_padded)
testing_labels_final= tf.stack(testing_labels_final)


In [19]:
# The GRU architecture
regressorGRU = Sequential()
# First GRU layer with Dropout regularisation
regressorGRU.add(GRU(units=50, return_sequences=True, input_shape=(X_train.shape[1],1), activation='tanh'))
regressorGRU.add(Dropout(0.2))
# Second GRU layer
regressorGRU.add(GRU(units=50, return_sequences=True, input_shape=(X_train.shape[1],1), activation='tanh'))
regressorGRU.add(Dropout(0.2))
# Third GRU layer
regressorGRU.add(GRU(units=50, return_sequences=True, input_shape=(X_train.shape[1],1), activation='tanh'))
regressorGRU.add(Dropout(0.2))
# Fourth GRU layer
regressorGRU.add(GRU(units=50, activation='tanh'))
regressorGRU.add(Dropout(0.2))
# The output layer
regressorGRU.add(Dense(units=1))
# Compiling the RNN
regressorGRU.compile(optimizer=SGD(lr=0.01, decay=1e-7, momentum=0.9, nesterov=False),loss='mean_squared_error')
# Fitting to the training set
regressorGRU.fit(X_train,y_train,epochs=1,batch_size=150)

/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


264/264 [==============================] - 318s 1s/step - loss: 0.2553


In [20]:
y_pred = regressorGRU.predict(X_test)
y_pred = (y_pred > 0.5)
#y_test_arg=np.argmax(y_test,axis=1)
#Y_pred = np.argmax(model.predict(X_test),axis=1)
pd.DataFrame(list(zip(y_test, y_pred)), columns=['Actual', 'Predicted'])

309/309 [==============================] - 34s 105ms/step


,Actual,Predicted
0,"[1.0, 0.0]",[False]
1,"[1.0, 0.0]",[True]
2,"[0.0, 1.0]",[False]
3,"[0.0, 1.0]",[False]
4,"[1.0, 0.0]",[True]
...,...,...
9877,"[1.0, 0.0]",[False]
9878,"[0.0, 1.0]",[False]
9879,"[0.0, 1.0]",[True]
9880,"[0.0, 1.0]",[False]


In [21]:
from sklearn.metrics import confusion_matrix, accuracy_score

#print(confusion_matrix(y_test, y_pred.round()))
#print("Accuracy:",accuracy_score(y_test, y_pred)*100,"%")


from sklearn.metrics import confusion_matrix, accuracy_score
print("Confusion Matrix:")
#con=confusion_matrix(y_test_, y_pred)
#rint(con)
#accuracy_score(y_true, y_pred.round(), normalize=False)
#print("Accuracy:",accuracy_score(y_test_, y_pred.round(),normalize=False)*100,"%")

y_test_arg=np.argmax(y_test,axis=1)
#Y_pred = np.argmax(model.predict(X_test),axis=1)

print(confusion_matrix(y_test_arg, y_pred))
print("Accuracy:",accuracy_score(y_test_arg, y_pred)*100,"%")

Confusion Matrix:
[[2953 2051]
 [2861 2017]]
Accuracy: 50.293462861768866 %


In [22]:
X_train.shape

(39528, 500)

In [23]:
model = Sequential() 
model.add(Dense(512, input_shape=(X_train.shape[1],), activation='relu', input_dim=3)) 